In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, RepeatVector, TimeDistributed

In [3]:
data = pd.read_csv("data_60_points.csv")
final_df = pd.read_csv("real_trainable_data.csv")

In [4]:
final_df.head()

,0,1,2,3,4,5,6,7,8,9,...,471,472,473,474,475,476,477,478,479,Target
0,24.292663,19.482136,8.624346,-0.932498,-0.395269,-0.455120,0.855522,0.265530,0.903650,0.375534,...,-17.311551,-22.457486,-28.753222,-27.479641,-28.066095,-25.329087,-24.605612,-24.266738,-21.633146,1.0
1,13.989555,10.538844,4.357757,-4.671976,-2.448575,-0.195571,0.800786,3.143261,5.239468,4.201113,...,-17.004589,-22.251523,-24.471177,-27.359055,-32.696794,-36.146539,-31.180339,-26.265950,-24.452572,2.0
2,28.226338,22.926922,12.568386,5.334923,5.037145,4.942274,5.231862,5.857457,4.706730,3.209744,...,-23.205840,-26.171166,-28.149451,-30.068256,-28.035475,-23.902512,-21.659425,-22.133117,-22.457960,3.0
3,10.275999,4.947967,-2.816720,-0.592784,3.641866,4.649932,2.646541,-1.270290,-2.929141,-6.799285,...,-23.478831,-25.957031,-28.022580,-31.121322,-33.086501,-26.612358,-23.061996,-22.466169,-24.421495,4.0
4,25.701159,19.995006,9.928850,2.699057,3.008508,3.815365,4.283556,5.071418,6.786370,5.908505,...,-16.841160,-16.545065,-18.070230,-22.589697,-27.299603,-27.420140,-24.701882,-23.104943,-23.844575,5.0


In [14]:
inputs = final_df.drop("Target", axis = 1)
inputs = inputs.iloc[:,0:60]
targets = final_df.Target 
targets = targets.apply(lambda x: int(x))

### Own Train and Validation Split

In [15]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(inputs, targets, test_size=0.15, random_state=42)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((20808, 60), (20808,), (3672, 60), (3672,))

### Normalization and Standarization 

In [22]:
from sklearn.preprocessing import Normalizer, MinMaxScaler
from sklearn.pipeline import Pipeline

# configure our pipeline
pipeline = Pipeline([('normalizer', Normalizer()),
                     ('scaler', MinMaxScaler())])

In [24]:
pipeline.fit(x_train)

x_train = pipeline.transform(x_train)
x_test = pipeline.transform(x_test)

## NN MODEL

In [25]:
model = Sequential()

model.add(Dense(64, activation = "relu", input_dim=x_train.shape[1]))
model.add(Dropout(0.5))
model.add(Dense(64, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "relu"))

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 64)                3904      
                                                                 
 dropout_4 (Dropout)         (None, 64)                0         
                                                                 
 dense_7 (Dense)             (None, 64)                4160      
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                                 
 dense_8 (Dense)             (None, 10)                650       
                                                                 
Total params: 8,714
Trainable params: 8,714
Non-trainable params: 0
_________________________________________________________________


In [26]:
model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics=['accuracy'])

In [27]:
history = model.fit(x_train, y_train, validation_data = [x_test, y_test], epochs = 20, batch_size = 64, verbose=2)

Epoch 1/20
326/326 - 3s - loss: nan - accuracy: 2.4029e-04 - val_loss: nan - val_accuracy: 0.0000e+00 - 3s/epoch - 9ms/step
Epoch 2/20
326/326 - 2s - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00 - 2s/epoch - 7ms/step
Epoch 3/20
326/326 - 2s - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00 - 2s/epoch - 7ms/step
Epoch 4/20
326/326 - 3s - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00 - 3s/epoch - 8ms/step
Epoch 5/20
326/326 - 2s - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00 - 2s/epoch - 8ms/step
Epoch 6/20
326/326 - 3s - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00 - 3s/epoch - 8ms/step
Epoch 7/20
326/326 - 3s - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00 - 3s/epoch - 10ms/step
Epoch 8/20
326/326 - 3s - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00 - 3s/epoch - 10ms/step
Epoch 

In [21]:
score = model.evaluate(x_test,y_test, batch_size=64)
score

58/58 [==============================] - 0s 5ms/step - loss: nan - accuracy: 0.0000e+00


[nan, 0.0]